装包

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install gast==0.2.2
!pip install folium==0.2.1
!pip install imgaug==0.2.5
!pip install kashgari==1.1.0

数据加载 训练测试集划分

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv(r"/content/drive/My Drive/KAGGLE/nlp/twitter/data/train.csv")

In [ ]:
raw_x=df["text"].values.tolist()
raw_y=df["target"].values.tolist()
raw_data=list(zip(raw_x,raw_y))

In [ ]:
training_data,test_data=train_test_split(raw_data,test_size=0.2, random_state=16)
training_data_x=[[w for w in sample[0].split()] for sample in training_data]
training_data_y=[sample[1] for sample in training_data]
test_data_x=[[w for w in sample[0].split()] for sample in test_data]
test_data_y=[sample[1] for sample in test_data]

In [ ]:
import kashgari
from kashgari.embeddings import BERTEmbedding
kashgari.config.use_cudnn_cell = True

In [ ]:
bert_embed = BERTEmbedding(r'/content/drive/My Drive/KAGGLE/nlp/twitter/bert/wwm_cased_L-24_H-1024_A-16',
                           task=kashgari.CLASSIFICATION,
                           sequence_length=100)

In [ ]:
from tensorflow.python import keras
import tensorflow as tf
import kashgari
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack
import logging
logging.basicConfig(level='DEBUG')
model = BiLSTM_Model(bert_embed)
eval_callback = EvalCallBack(kash_model=model,valid_x=test_data_x,valid_y=test_data_y,step=1)
model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=10,epochs=10,callbacks=[eval_callback])

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 250)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 250)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 250, 1024),  29691904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 250, 1024)    2048        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
model.evaluate(test_data_x,test_data_y,debug_info=True)

              precision    recall  f1-score   support

           0     0.8042    0.7486    0.7754       867
           1     0.6955    0.7591    0.7259       656

    accuracy                         0.7531      1523
   macro avg     0.7499    0.7539    0.7507      1523
weighted avg     0.7574    0.7531    0.7541      1523



In [ ]:
from tensorflow.python import keras
from kashgari.tasks.classification import DPCNN_Model
import tensorflow as tf
from kashgari.callbacks import EvalCallBack
#hyper = DPCNN_Model.get_default_hyper_parameters()
#hyper['rnn_0']['units'] = 250
#hyper['rnn_1']['units'] = 250
model = DPCNN_Model(bert_embed)#,hyper_parameters=hyper)
# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=1)
early_stop = keras.callbacks.EarlyStopping(patience=100)
model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=16,epochs=20,callbacks=[eval_callback,early_stop])

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 1024),  29691904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 1024)    2048        Input-Segment[0][0]              
___________________________________________________________________________________________

In [ ]:
from tensorflow.python import keras
from kashgari.tasks.classification import CNN_LSTM_Model

from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = CNN_LSTM_Model()

eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=1)
early_stop = keras.callbacks.EarlyStopping(patience=10)
reduse_lr_callback = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5)
# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=5)

model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=10,epochs=30,callbacks=[eval_callback,early_stop,reduse_lr_callback])

In [ ]:
model.evaluate(test_data_x,test_data_y,debug_info=True)